In [ ]:
import cv2
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from statistics import stdev,mean

In [ ]:
#define globals required through out the whole program
edges=[] #containing all edge tuple
attrs=[] #countaining list of attribute of all nodes
graph_id=1 #id of latest graph
node_id=1 #id of latest node
graph_indicator=[] #containing graph-id for each node
node_labels=[] #containing labels for all node
graph_labels=[]#containing labels for all graph

#activity-label vs activity-name mapping (4-class)
activity_map={}
activity_map[1]='BIRAD_0'
activity_map[2]='BIRAD_1'
activity_map[3]='BIRAD_2'
activity_map[4]='BIRAD_3'
activity_map[5]='BIRAD_4A'
activity_map[6]='BIRAD_4B'
activity_map[7]='BIRAD_4C'
activity_map[8]='BIRAD_5'

In [ ]:
#z-score normalization
def normalize(arr):
    arr=np.array(arr)
    m=np.mean(arr)
    s=np.std(arr)
    return (arr-m)/s

#generate graph for a given edge-image file
def generate_graphs(filename,node_label,activity_map):
    print(" ... Reading image: "+filename+" ...")
    global node_id,edges,attrs,graph_id,node_labels,graph_indicator
    cnt=0
    img=cv2.imread(filename)
    dim1,dim2,_=img.shape
    attrs1=[]

    print("Image type: " + activity_map[node_label] + "\nPixel matrix is of: "+str(dim1)+"x"+str(dim2))
    img1=img.copy()
    nodes=np.full((dim1,dim2),-1)
    edge=0
    for i in range(dim1):
        for j in range(dim2):
        #considering pixel as node if pixel-value>=128
        b,_,_=img[i][j]
        if(b>=128):
            nodes[i][j]=node_id
            attrs1.append(b)
            graph_indicator.append(graph_id)
            node_labels.append([node_label,activity_map[node_label]])
            node_id+=1
            cnt+=1
        else:
            img1[i][j]=0
  
    for i in range(dim1):
        for j in range(dim2):
            #forming edge between all adjacent pixels which are node
            if(nodes[i][j]!=-1):
                li=max(0,i-1)
                ri=min(i+2,dim1)
                lj=max(0,j-1)
                rj=min(j+2,dim2)
            for i1 in range(li,ri):
                for j1 in range(lj,rj):
                    if((i1!=i or j1!=j) and (nodes[i1][j1]!=-1)):
                        edges.append([nodes[i][j],nodes[i1][j1]])
                        edge+=1
  
    attrs1=normalize(attrs1)
    attrs.extend(attrs1)
    del attrs1
    print("For given image nodes formed: "+str(cnt)+" edges formed: "+str(edge))
    if(cnt!=0): graph_id+=1


In [ ]:
#generate graphs for all edge-image under given dir along with proper label
def generate_graph_with_labels(dirname,label,activity_map):
    print("\n... Reading Directory: "+dirname+" ...\n")
    global graph_labels
    filenames=glob.glob(dirname+'/*.jpg')
    for filename in filenames:
        generate_graphs(filename,label,activity_map)
        graph_labels.append([label,activity_map[label]])

#generate graphs for all directories
def process_graphs(BIRAD_0_dir,
                   BIRAD_1_dir,
                   BIRAD_2_dir,
                   BIRAD_3_dir,
                   BIRAD_4A_dir,
                   BIRAD_4B_dir,
                   BIRAD_4C_dir,
                   BIRAD_5_dir,
                   activity_map):
    global node_labels,graph_labels
    generate_graph_with_labels(BIRAD_0_dir, 1, activity_map)
    generate_graph_with_labels(BIRAD_1_dir, 2, activity_map)
    generate_graph_with_labels(BIRAD_2_dir, 3, activity_map)
    generate_graph_with_labels(BIRAD_3_dir, 4, activity_map)
    generate_graph_with_labels(BIRAD_4A_dir, 5, activity_map)
    generate_graph_with_labels(BIRAD_4B_dir, 6, activity_map)
    generate_graph_with_labels(BIRAD_4C_dir, 7, activity_map)
    generate_graph_with_labels(BIRAD_5_dir, 8, activity_map)
    print("Processing done")
    print("Total nodes formed: "+str(len(node_labels))+"Total graphs formed: "+str(len(graph_labels)))




In [ ]:
#working directories
BIRAD_0_dir = 'Mammograms/data/BIRAD_0'
BIRAD_1_dir = 'Mammograms/data/BIRAD_1'
BIRAD_2_dir = 'Mammograms/data/BIRAD_2'
BIRAD_3_dir = 'Mammograms/data/BIRAD_3'
BIRAD_4A_dir = 'Mammograms/data/BIRAD_4A'
BIRAD_4B_dir = 'Mammograms/data/BIRAD_4B'
BIRAD_4C_dir = 'Mammograms/data/BIRAD_4C'
BIRAD_5_dir = 'Mammograms/data/BIRAD_5'

process_graphs(BIRAD_0_dir,
               BIRAD_1_dir,
               BIRAD_2_dir,
               BIRAD_3_dir,
               BIRAD_4A_dir,
               BIRAD_4B_dir,
               BIRAD_4C_dir,
               BIRAD_5_dir,
               activity_map)

#check all the lengths of globals
#comment if not necessary
print(len(node_labels))
print(len(graph_labels))
print(len(edges))
print(len(attrs))

#create adjacency dataframe
df_A=pd.DataFrame(columns=["node-1","node-2"],data=np.array(edges))
print("Shape of edge dataframe: "+str(df_A.shape))
print("\n--summary of dataframe--\n" ,df_A.head(50))

#create node label dataframe
df_node_label=pd.DataFrame(data=np.array(node_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_node_label.shape))
print("\n--summary of dataframe--\n" ,df_node_label)

#create graph label dataframe
df_graph_label=pd.DataFrame(data=np.array(graph_labels),columns=["label","activity-name"])
print("shape of node-label dataframe: "+str(df_graph_label.shape))
print("\n--summary of dataframe--\n" ,df_graph_label.head(50))

#create node-attribute dataframe (normalized grayscale value)
df_node_attr=pd.DataFrame(data=np.array(attrs),columns=["gray-val"])
print("shape of node-attribute dataframe: "+str(df_node_attr.shape))
print("\n--summary of dataframe--\n" ,df_node_attr.head(50))

#create graph-indicator datframe
df_graph_indicator=pd.DataFrame(data=np.array(graph_indicator),columns=["graph-id"])
print("shape of graph-indicator dataframe: "+str(df_graph_indicator.shape))
print("\n--summary of dataframe--\n" ,df_graph_indicator.head(50))

#omit activity name later for graph-label and node-label
#since GIN model will only accept the label
df_node_label=df_node_label.drop(["activity-name"],axis=1)
print(df_node_label.head(50))

df_graph_label=df_graph_label.drop(["activity-name"],axis=1)
print(df_graph_label.head(50))



def save_dataframe_to_txt(df,filepath):
    df.to_csv(filepath,header=None,index=None,sep=',',mode='w')



#save all the dataframes to .txt file
#path name: .../GraphTrain/dataset/<dataset_name>/raw/<dataset_name>_<type>.txt
# <type>:
# A--> adjancency matrix
#graph_indicator--> graph-ids of all node
#graph_labels--> labels for all graph
#node_attributes--> attribute(s) for all node
#node_labels--> labels for all node

sourcepath='Mammograms/data/raw'
save_dataframe_to_txt(df_A, sourcepath + '/8class_A.txt')
save_dataframe_to_txt(df_graph_indicator, sourcepath+'/8class_graph_indicator.txt')
save_dataframe_to_txt(df_graph_label, sourcepath+'/8class_graph_labels.txt')
save_dataframe_to_txt(df_node_attr, sourcepath+'/8class_node_attributes.txt')
save_dataframe_to_txt(df_node_label, sourcepath+'/8class_node_labels.txt')